In [4]:
#importing libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

Using TensorFlow backend.


In [5]:
#Initialize CNN model
model=Sequential()

In [6]:
#Add Covolution Layer
model.add(Conv2D(32,3,3,input_shape=(128,128,3),activation="relu"))#32 no:of feature detectors to apply in the image
#3,3 is the size of feature detector
#input_shape is size of image

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., activation="relu")`
  from ipykernel import kernelapp as app


In [7]:
#Add Pooling Layer
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,3,3,activation="relu"))#32 no:of feature detectors to apply in the image
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,3,3,activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  app.launch_new_instance()
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`


In [8]:
#Add Flattening Layer
model.add(Flatten())

In [9]:
#Add Hidden Layer
model.add(Dense(init="uniform",activation="relu",output_dim=256))
model.add(Dense(init="uniform",activation="relu",output_dim=128))


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", units=256)`
  from ipykernel import kernelapp as app
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", units=128)`
  app.launch_new_instance()


In [10]:
#Add Output Layer
model.add(Dense(init="uniform",activation="softmax",output_dim=3))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="softmax", units=3)`
  from ipykernel import kernelapp as app


In [11]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=['accuracy'])

In [12]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_27772a874b4648048341f2979d4ee10e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Uqhg9GeY_tctNxER5Uab1GL-HJkLOwfSp_YxGYl9otXD',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
streaming_body_2 = client_27772a874b4648048341f2979d4ee10e.get_object(Bucket='pneumoniaprediction-donotdelete-pr-fhyxpzswlzqsll', Key='Pneumonia.zip')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(streaming_body_2, "__iter__"): streaming_body_2.__iter__ = types.MethodType( __iter__, streaming_body_2 ) 



In [16]:
from io import BytesIO
import zipfile

zip_ref = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
paths = zip_ref.namelist()
classes_required=[]
for path in paths:
    zip_ref.extract(path)
    temp=path.split('/')
    if len(temp) > 3:
        if temp[2] not in classes_required:
            classes_required.append(temp[2])
print(classes_required)
zip_ref.close()

['chest_xray', 'test', 'train']


In [17]:
import os
cwd = os.getcwd()
cwd

'/home/dsxuser/work'

In [18]:
x_train = train_datagen.flow_from_directory(
        r'/home/dsxuser/work/Pneumonia/chest_xray/train',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')


Found 5315 images belonging to 3 classes.


In [19]:
x_test = test_datagen.flow_from_directory(
        r'/home/dsxuser/work/Pneumonia/chest_xray/test',
        target_size=(128,128),
        batch_size=32,
        class_mode='categorical')


Found 652 images belonging to 3 classes.


In [17]:
#training the model
model.fit_generator(
        x_train,
        steps_per_epoch=166,
        epochs=30,
        validation_data=x_test,
        validation_steps=21)

Epoch 1/30
166/166 [==============================] - 536s 3s/step - loss: 0.4866 - acc: 0.7986 - val_loss: 0.5264 - val_acc: 0.8083
Epoch 2/30
166/166 [==============================] - 542s 3s/step - loss: 0.2409 - acc: 0.9002 - val_loss: 0.5039 - val_acc: 0.8267
Epoch 3/30
166/166 [==============================] - 540s 3s/step - loss: 0.2284 - acc: 0.9080 - val_loss: 0.2889 - val_acc: 0.8850
Epoch 4/30
166/166 [==============================] - 540s 3s/step - loss: 0.2204 - acc: 0.9115 - val_loss: 0.3963 - val_acc: 0.8696
Epoch 5/30
166/166 [==============================] - 540s 3s/step - loss: 0.1893 - acc: 0.9268 - val_loss: 0.3734 - val_acc: 0.8758
Epoch 6/30
166/166 [==============================] - 532s 3s/step - loss: 0.1826 - acc: 0.9292 - val_loss: 0.3426 - val_acc: 0.9003
Epoch 7/30
166/166 [==============================] - 529s 3s/step - loss: 0.1590 - acc: 0.9354 - val_loss: 0.3193 - val_acc: 0.8911
Epoch 8/30
166/166 [==============================] - 523s 3s/step - 

In [77]:
model.save("cnn_project.h5")

In [78]:
x_train.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1, 'negative': 2}

In [79]:
get_ipython().system(u'tar -zcvf cnn_project.tgz cnn_project.h5')

cnn_project.h5


In [80]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [81]:
wml_credentials={
  "url": "https://eu-gb.ml.cloud.ibm.com",
  "access_key": "sLcPkqv4vZYjnTeXWwWbJyfYQZ6K0pauFaiJ6byBMSPZ",
  "username": "2af079c0-ee7d-416a-9e62-fd1697c406ce",
  "password": "0aeae914-dac0-4584-8848-b0e72d512af6",
  "instance_id": "f035d7a8-e509-426d-8f40-07fcc74886a7"
}

In [82]:
client = WatsonMachineLearningAPIClient( wml_credentials )

In [92]:
metadata = {
    client.repository.ModelMetaNames.NAME: "keras model",
    client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES: [{'name':'keras', 'version': '2.1.3'}],
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5"
}

In [93]:
model_details = client.repository.store_model( model="cnn_project.tgz", meta_props=metadata )

In [94]:
client.deployments.list()

------------------------------------  ---------  ------  --------------  ------------------------  --------------  -------------
GUID                                  NAME       TYPE    STATE           CREATED                   FRAMEWORK       ARTIFACT TYPE
8ee16deb-72c9-45ea-8688-a552229e198d  Pneumonia  online  DEPLOY_FAILURE  2019-05-24T04:22:05.448Z  tensorflow-1.5  model
------------------------------------  ---------  ------  --------------  ------------------------  --------------  -------------


In [95]:
model_id = model_details["metadata"]["guid"]
model_deployment_details = client.deployments.create( artifact_uid=model_id, name="Pneumonia" )



#######################################################################################

Synchronous deployment creation for uid: '55f60f78-2d6a-4d00-bda3-13e4bcc5bb46' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='0bfc3491-c136-4c8f-b82e-c8c6d2da10b2'
------------------------------------------------------------------------------------------------




In [96]:
scoring_endpoint = client.deployments.get_scoring_url(model_deployment_details)
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/f035d7a8-e509-426d-8f40-07fcc74886a7/deployments/0bfc3491-c136-4c8f-b82e-c8c6d2da10b2/online'